## Set-up and installations

In [ ]:
# Python2 kernel
%%bash
pip install anndata2ri
conda install seaborn scikit-learn statsmodels numba pytables
conda install -c conda-forge python-igraph leidenalg

## Import Treg data from R to Python

In [ ]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -o adata_sce
library(scran)
sce <- readRDS(file = "./data/sce.rds")
sce_sub = sce[,which(colData(sce)$macro_label == "Treg")]
adata_sce <- as(sce_sub, 'SingleCellExperiment')
print(adata_sce)

## PAGA settings 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParamsOrig
import scanpy as sc

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
results_file = './write/paga.h5ad'
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(3, 3), facecolor='white')  # low dpi (dots per inch) yields small inline figures

In [ ]:
adata_sce.X = adata_sce.X.astype('float64')  # this is not required and results will be comparable without it

## Preprocess Treg data with standard PAGA pipeline

In [ ]:
sc.pp.recipe_zheng17(adata_sce)

In [ ]:
adata_sce

## PAGA

### Diffusion Map for dimensionality reduction

In [ ]:
sc.pp.neighbors(adata_sce, n_neighbors = 30, n_pcs =20)
#sc.tl.diffmap(adata_sce)

In [ ]:
#sc.pp.neighbors(adata_sce, n_neighbors=5, use_rep='X_diffmap')
sc.tl.draw_graph(adata_sce)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 6]
sc.pl.draw_graph(adata_sce, color='cluster_update_label', legend_loc='on data')

### Compute PAGA graph

In [ ]:
sc.tl.paga(adata_sce, groups='cluster_update_label')
sc.pl.paga(adata_sce, color='cluster_update_label')

In [ ]:
print('Outputting cluster edges...')

import os
def con2edges(con, names=None, sparse=True):
    print('Converting connectivity matrix to edges...')
    n = con.shape[0]
    edges = pd.DataFrame(columns=['From', 'To', 'Connectivity'])

    for i in range(n):
        for j in range(i + 1, n):
            if names is not None:
                fr = names[i]
                to = names[j]
            else:
                fr = str(i)
                to = str(j)

            connectivity = con[i, j]
            if sparse and connectivity == 0:
                continue

            entry = {'From' : fr, 'To' : to,
                     'Connectivity' : con[i, j]}
            edges = edges.append(entry, ignore_index=True)

    return edges

clust_con = adata_sce.uns['paga']['connectivities'].toarray()
clust_edges = con2edges(clust_con, names = ['aTreg', 'Tfr', 'OxTreg', 'ST2 Treg', 'ST2 mTreg', 'mTreg'])
clust_edges.to_csv(os.path.join("./output/paga/", 'cluster_edges.csv'),
                  index=False)

print('Outputting cluster embedding...')
clust_embedding = pd.DataFrame(adata_sce.uns['paga']['pos'], columns=['X', 'Y'])
#clust_embedding['cluster_update_label'] = range(clust_embedding.shape[0])
clust_embedding['cluster_update_label'] = ['aTreg', 'Tfr', 'OxTreg', 'ST2 Treg', 'ST2 mTreg', 'mTreg']
clust_embedding = clust_embedding[['cluster_update_label', 'X', 'Y']]
clust_embedding.to_csv(os.path.join("/home/chsiao/IL2_timecourse/output/paga/", 'cluster_embedding.csv'),
                       index=False)

print('Outputting cell embedding...')
x = adata_sce.obsm['X_draw_graph_fa'][:, 0]
y = adata_sce.obsm['X_draw_graph_fa'][:, 1]
cell_embedding = pd.DataFrame({'Cell' : cells, 'X' : x, 'Y' : y})
cell_embedding.to_csv(os.path.join("./output/paga/", 'cell_embedding.csv'),
                      index=False)

In [ ]:
%%R
library(tidyverse)
clust_edges = read.table("./output/paga/cluster_edges.csv", header = TRUE, sep = ",")
plot_data <- tibble(
    Threshold = seq(0, 1, 0.01)
) %>%
    mutate(Edges = map_int(Threshold, function(thresh) {
        sum(clust_edges$Connectivity > thresh)
    }))

con_thresh <- 0.5

ggplot(plot_data, aes(x = Threshold, y = Edges)) +
    geom_point() +
    geom_line() +
    geom_vline(xintercept = con_thresh, colour = "red") +
    xlab("Connectivity threshold") +
    ylab("Number of edges") +
    theme_minimal()

In [ ]:
%%R
library(tidyverse)
library(knitr)
cluster_edges = read.table("./output/paga/cluster_edges.csv", header = TRUE, sep = ",")
cluster_embedding = read.table("./output/paga/cluster_embedding.csv", header = TRUE, sep = ",")

plotPAGAClustGraph <- function(embedding, edges, thresh = 0,
                               colour = "cluster_update_label") {

    is_discrete <- is.factor(embedding[[colour]])

    gg <- ggplot(embedding, aes(x = X, y = Y))

    if (is_discrete) {
        gg <- gg +
            geom_segment(data = filter(edges, Connectivity > thresh),
                         aes(x = FromX, y = FromY, xend = ToX, yend = ToY,
                             colour = Connectivity),
                         size = 4) +
            scale_colour_viridis_c(limits = c(0, 1))
    } else {
        gg <- gg +
            geom_segment(data = filter(edges, Connectivity > thresh),
                         aes(x = FromX, y = FromY, xend = ToX, yend = ToY,
                             alpha = Connectivity),
                         size = 4, colour = "grey30") +
            scale_alpha(limits = c(0, 1)) +
            scale_fill_viridis_c()
    }

    gg <- gg +
        geom_point(aes(fill = !!ensym(colour), size = Size), shape = 21) +
        geom_text(aes(label = Cluster)) +
        scale_size(range = c(5, 15)) +
        theme_void() +
        theme(legend.position = "none")

    return(gg)
}
#src_list <- lapply(seq(0, 0.9, 0.1), function(thresh) {
#    src <- c(
#        "### Con {{thresh}} {.unnumbered}",
#        "```{r clust-paga-{{thresh}}}",
#        "plotPAGAClustGraph(clust_embedding, clust_edges, thresh = {{thresh}})",  
#        "```",
#        ""
#    )
#    knit_expand(text = src)
#})
plotPAGAClustGraph(cluster_embedding, cluster_edges, thresh = 0.1)

#out <- knit_child(text = unlist(src_list), options = list(cache = FALSE))

Recompute the graph using PAGA initialization

In [ ]:
sc.pl.paga(adata_sce, color='cluster_update_label', threshold = 0.2)
plt.savefig("./figs/paga.eps', format='eps')

In [ ]:
%%R
library(igraph)
library(tidyverse)
library(knitr)
links = read.table("./output/paga/cluster_edges.csv", header = TRUE, sep = ",", as.is = TRUE)

links <- aggregate(cluster_edges[,3], cluster_edges[,-3], sum)
links <- links[order(links$From, links$To),]
colnames(links)[3] <- "weight"
rownames(links) <- NULL

nodes = data.frame(id = c("aTreg", "Tfr", "OxTreg", "ST2 Treg", "ST2 mTreg", "mTreg"))

links = links[links$weight > .2,]
net <- graph_from_data_frame(d=links, vertices = nodes, directed = F) 

cols = RColorBrewer::brewer.pal(n = 8, name = 'Dark2')
Treg_cols = c(cols[2], cols[6], cols[3], cols[4], cols[5], cols[1])

cairo_ps(filename = "/home/chsiao/IL2_timecourse/figs/paper/paga.eps",
         width = 4, height = 4, pointsize = 8,
         fallback_resolution = 300)
plot(net, vertex.label.dist=2.1, 
     vertex.color = Treg_cols, 
     vertex.label.color = "black", 
     vertex.label.cex = 1.2, vertex.label.family = "Arial",
     edge.label = round(links$weight,2),
     edge.label.color = "black", edge.label.family = "Arial",
     edge.width = round(links$weight,2)*8)
dev.off()
#net$vertex.label

In [ ]:
sc.tl.draw_graph(adata_sce, init_pos='paga')

In [ ]:
sc.pl.draw_graph(adata_sce, color=['cluster_update_label'], legend_loc='on data')

In [ ]:
sc.pl.embedding_density(adata_sce, basis = "umap", group = 'cluster_update_label')

In [ ]:
for condition_day in ['utx_0', 'WT_2', 'WT_4', 'TM88_2', 'TM88_4']:
    sc.pl.draw_graph(adata_sce, color=['cluster_update_label', 'Stmn1', 'Gzmb'], legend_loc='on data', groups = [condition_day])

In [ ]:
# choose a root cell for diffusion pseudotime
adata_sce.uns['iroot'] = np.flatnonzero(adata_sce.obs['cluster_update_label']  == 'aTreg')[0]

In [ ]:
sc.tl.dpt(adata_sce)

In [ ]:
gene_names = ['Stmn1', 'Nusap1', 'Gzmb', 'Top2a', 'Ccna2',  
              'Ikzf2', 'Gpr83', 'Lrrc32',                   
              'Atp5c1', 'Ndufv3', 'Cox4i1',
              'Egr2', 'Bcl6', 'Il1rl1', 'Cxcr5', 'Icox'] 

In [ ]:
sc.pl.draw_graph(adata_sce, color=['cluster_update_label', 'dpt_pseudotime'], legend_loc='on data')

In [ ]:
#for condition_day in ['utx_0']:
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'utx_0'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "utx_0")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_2'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "WT_2")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_2'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "TM88_2")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_4'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "WT_4")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_4'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "TM88_4")

In [ ]:
# choose a root cell for diffusion pseudotime
adata_sce.uns['iroot'] = np.flatnonzero(adata_sce.obs['cluster_update_label']  == 'Tfr')[0]

In [ ]:
sc.tl.dpt(adata_sce)

In [ ]:
sc.pl.draw_graph(adata_sce, color=['cluster_update_label', 'dpt_pseudotime'], legend_loc='on data')

In [ ]:
#for condition_day in ['utx_0']:
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'utx_0'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "utx_0")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_2'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "WT_2")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_2'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "TM88_2")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_4'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "WT_4")
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_4'], color=['cluster_update_label','dpt_pseudotime'], legend_loc='on data', title = "TM88_4")

In [ ]:
sc.tl.embedding_density(adata_sce, groupby='cluster_update_label')

In [ ]:
sc.pl.embedding_density(adata_sce, groupby='cluster_update_label')

In [ ]:
#sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'utx_0'], color=['Stmn1', 'Gzmb'], legend_loc='on data')
#sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_2'], color=['Stmn1', 'Gzmb'], legend_loc='on data')
sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_2'], color=['Stmn1', 'Gzmb'], legend_loc='on data')
#sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'WT_4'], color=['Stmn1', 'Gzmb'], legend_loc='on data', title = "WT_4")
#sc.pl.draw_graph(adata_sce[adata_sce.obs['condition_day'] == 'TM88_4'], color=['Stmn1', 'Gzmb'], legend_loc='on data', title = "TM88_4")